In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import random
from itertools import chain, combinations, tee
import time


# General CBCP Equilibrium Solver

## (Special Case) Quartic Polynomial Latency Functions

In [2]:
# grad = np.array([0, 1, 2, 3, 4])
grad = np.array([2, 4, 0, 1, 3])

for id_temp, entry_temp in enumerate(grad):
    print("id_temp, entry_temp:", id_temp, entry_temp)


id_temp, entry_temp: 0 2
id_temp, entry_temp: 1 4
id_temp, entry_temp: 2 0
id_temp, entry_temp: 3 1
id_temp, entry_temp: 4 3


In [23]:
def welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array, v_E_array, y_el, y_in, \
                a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    ## Variable indices:
    # In full:
    # y_el indices: (group, edge, "lane", time)
    # y_in indices: (group, edge, "lane", time)
    # Here:
    # y_el indices: (group, "lane")
    # y_in indices: (group, "lane")
    
    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]
    
    ## Compute lane flows:
    
    # Express lane (ex):
    x_ex = 0.0
    x_ex += sum(y_el[(g, k)] for g in range(num_el) for k in [0, 1])
    x_ex += sum(y_in[(g, 0)] for g in range(num_in))
    
    # General purpose lane (gp):
    x_gp = 0.0 
    x_gp += sum(y_el[(g, 2)] for g in range(num_el))
    x_gp += sum(y_in[(g, 1)] for g in range(num_in))
    
    ell_ex = sum(a_ex[p] * (x_ex ** p) for p in range(5))
    ell_gp = sum(a_gp[p] * (x_gp ** p) for p in range(5))
    
    obj_E = tau * sum(y_el[(g, 1)] for g in range(num_el)) \
                + sum(y_el[(g, k)] * v_E_array[g] for g in range(num_el) for k in [0, 1]) * ell_ex \
                + sum(y_el[(g, 2)] * v_E_array[g] for g in range(num_el)) * ell_gp
    obj_I = tau * sum(y_in[(g, 0)] for g in range(num_in)) \
                + sum(y_in[(g, 0)] * v_I_array[g] for g in range(num_in)) * ell_ex \
                + sum(y_in[(g, 1)] * v_I_array[g] for g in range(num_in)) * ell_gp  
    obj_R = tau * sum(y_el[(g, 1)] for g in range(num_el)) \
                + tau * sum(y_in[(g, 0)] for g in range(num_in))
    
    welfare = lambda_E * obj_E - lambda_R * obj_R + lambda_I * obj_I
    
    print()
    print("obj_E:", obj_E)
    print("obj_R:", obj_R)
    print("obj_I:", obj_I)
    print("welfare:", welfare)
    print()

    return welfare

def latencies(tau, v_I_array, v_E_array, y_el, y_in, \
                  a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                  a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):

    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]
    
    ## Compute lane flows:
    
    # Express lane (ex):
    x_ex = 0.0
    y_el_ex = 0.0
    y_in_ex = 0.0
    
    x_ex += sum(y_el[(g, k)] for g in range(num_el) for k in [0, 1])
    x_ex += sum(y_in[(g, 0)] for g in range(num_in))
    y_el_ex += sum(y_el[(g, k)] for g in range(num_el) for k in [0, 1])
    y_in_ex += sum(y_in[(g, 0)] for g in range(num_in))
    
    # General purpose lane (gp):
    x_gp = 0.0 
    y_el_gp = 0.0
    y_in_gp = 0.0
    
    x_gp += sum(y_el[(g, 2)] for g in range(num_el))
    x_gp += sum(y_in[(g, 1)] for g in range(num_in))
    y_el_gp += sum(y_el[(g, 2)] for g in range(num_el))
    y_in_gp += sum(y_in[(g, 1)] for g in range(num_in))
    
    ell_ex = sum(a_ex[p] * (x_ex ** p) for p in range(5))
    ell_gp = sum(a_gp[p] * (x_gp ** p) for p in range(5))
        
    total_latency = x_ex * ell_ex + x_gp * ell_gp    
    el_latency = y_el_ex * ell_ex + y_el_gp * ell_gp
    in_latency = y_in_ex * ell_ex + y_in_gp * ell_gp
    
    assert abs(total_latency - in_latency - el_latency) < 1E-4, "We should have total_latency == in_latency + el_latency"
    
    return total_latency, el_latency, in_latency

In [4]:
def proj_tau_B_11(tau, B, tau_max = 1.0, B_max = 1.0):
    
    tau_feas = cp.Variable(1)
    B_feas = cp.Variable(1)
    
    func = (tau_feas - tau)**2 + (B_feas - B)**2

    objective = cp.Minimize(func)
    
    constraints = []
#     constraints += [0 <= tau_feas <= 1]
#     constraints += [0 <= B_feas <= 1]
    constraints += [tau_feas >= 0.0]
    constraints += [B_feas >= 0.0]
    constraints += [tau_feas <= tau_max]
    constraints += [B_feas <= B_max]
    constraints += [B_feas <= tau_feas]
    
    prob = cp.Problem(objective, constraints)
    result = prob.solve()

    return np.round(np.maximum(tau_feas.value, 0.0), decimals=4), \
            np.round(np.maximum(B_feas.value, 0.0), decimals=4)


# Counterexample, Budget Increase also Increases Eligible Users' Cost

In [24]:

# Below: For quartic latency functions:
# Latency Function: a_4 x^4 + a_3 x^3 + a_2 x^2 + a_1 x + a_0
def solve_CBCP_direct(tau, B, v_I_array, v_E_array, \
                         a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                         a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    ## Variable indices:
    # In full:
    # y_el indices: (group, edge, "lane", time)
    # y_in indices: (group, edge, "lane", time)
    # Here:
    # y_el indices: (group, "lane")
    # y_in indices: (group, "lane")

    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]

    # Variables:
    y_el = {}
    for g in range(num_el):
        for k in [0, 1, 2]:
            y_el[(g, k)] = cp.Variable(1)
            
    y_in = {}
    for g in range(num_in):
        for k in [0, 1]:
            y_in[(g, k)] = cp.Variable(1)
            
    x_ex = cp.Variable(1)
    x_gp = cp.Variable(1)
    
    # Objective:
    func = 0.0
    
    func += 1/5 * a_ex[4] * cp.power(x_ex, 5)
    func += 1/4 * a_ex[3] * cp.power(x_ex, 4)
    func += 1/3 * a_ex[2] * cp.power(x_ex, 3)
    func += 1/2 * a_ex[1] * cp.power(x_ex, 2)
    func += a_ex[0] * x_ex
    func += 1/5 * a_gp[4] * cp.power(x_gp, 5)
    func += 1/4 * a_gp[3] * cp.power(x_gp, 4)
    func += 1/3 * a_gp[2] * cp.power(x_gp, 3)
    func += 1/2 * a_gp[1] * cp.power(x_gp, 2)
    func += a_gp[0] * x_gp
    
    func += sum(y_el[(g, 1)] / v_E_array[g] for g in range(num_el)) * tau
    func += sum(y_in[(g, 0)] / v_I_array[g] for g in range(num_in)) * tau

    objective = cp.Minimize(func)
    
    # Constraints:
    constraints = []
    
    constraints += [y_el[(g, k)] >= 0.0 for g in range(num_el) for k in [0, 1, 2]]
    constraints += [y_in[(g, k)] >= 0.0 for g in range(num_in) for k in [0, 1]]
    
    constraints += [sum(y_el[(g, k)] for k in [0, 1, 2] ) == 1.0 for g in range(num_el)]
    constraints += [sum(y_in[(g, k)] for k in [0, 1] ) == 1.0 for g in range(num_in)]
    
    constraints += [x_ex == sum(y_el[(g, k)] for g in range(num_el) for k in [0, 1]) \
                               + sum(y_in[(g, 0)] for g in range(num_in))]
    constraints += [x_gp == sum(y_el[(g, 2)] for g in range(num_el)) \
                               + sum(y_in[(g, 1)] for g in range(num_in))]
    
    constraints += [y_el[(g, 0)] * tau <= B for g in range(num_el)]
    
    # Problem:
    prob = cp.Problem(objective, constraints)
    
    # Solve:
    result = prob.solve()
    
    # Extract Values:
    y_el_values = {}
    for g in range(num_el):
        for k in [0, 1, 2]:
            y_el_values[(g, k)] = np.round(np.max(y_el[(g, k)].value[0]), decimals = 4)
            
    y_in_values = {}
    for g in range(num_in):
        for k in [0, 1]:
            y_in_values[(g, k)] = np.round(np.max(y_in[(g, k)].value[0]), decimals = 4)

    return y_el_values, y_in_values


In [28]:
a = np.array([0.0, 1.0, 0.0, 0.0, 0.0])

tau = 4.0
B = 0.15 * (4**2)
v_I_array = np.array([100.0, 1.5, 1.4])
v_E_array = np.array([1.0])

y_el_values, y_in_values = solve_CBCP_direct(tau = tau, B = B, v_I_array = v_I_array, \
                                             v_E_array = v_E_array, a_ex = a, a_gp = a)

total_latency, el_latency, in_latency = latencies(tau, v_I_array, v_E_array, \
                                                  y_el = y_el_values, y_in = y_in_values, \
                                                  a_ex = a, a_gp = a)

print("y_el_values:", y_el_values)
print("y_in_values:", y_in_values)
print()

print("total_latency:", total_latency)
print("el_latency:", el_latency)
print("in_latency:", in_latency)


y_el_values: {(0, 0): 0.6, (0, 1): -0.0, (0, 2): 0.4}
y_in_values: {(0, 0): 1.0, (0, 1): 0.0, (1, 0): 0.0, (1, 1): 1.0, (2, 0): 0.0, (2, 1): 1.0}

total_latency: 8.32
el_latency: 1.92
in_latency: 6.4


In [29]:
a = np.array([0.0, 1.0, 0.0, 0.0, 0.0])

tau = 4.0
B = 0.24 * (4**2)
v_I_array = np.array([100.0, 1.5, 1.4])
v_E_array = np.array([1.0])

y_el_values, y_in_values = solve_CBCP_direct(tau = tau, B = B, v_I_array = v_I_array, \
                                             v_E_array = v_E_array, a_ex = a, a_gp = a)

total_latency, el_latency, in_latency = latencies(tau, v_I_array, v_E_array, \
                                                  y_el = y_el_values, y_in = y_in_values, \
                                                  a_ex = a, a_gp = a)

print("y_el_values:", y_el_values)
print("y_in_values:", y_in_values)
print()

print("total_latency:", total_latency)
print("el_latency:", el_latency)
print("in_latency:", in_latency)
print()


y_el_values: {(0, 0): 0.96, (0, 1): -0.0, (0, 2): 0.04}
y_in_values: {(0, 0): 1.0, (0, 1): 0.0, (1, 0): 0.0, (1, 1): 1.0, (2, 0): 0.0, (2, 1): 1.0}

total_latency: 8.0032
el_latency: 1.9632
in_latency: 6.04



## Test:

In [ ]:
# Test:

grad = np.array([3.11430535, 1.501, 1.501, 2.46858321, 1.501])

# y_el: \hat y_1 E, \tilde y_1 E, y_2 E
y_el_var = cp.Variable(3)
# y_in: y_1 I, y_2 I
y_in_var = cp.Variable(2)

objective = cp.Minimize(grad[0:3] @ y_el_var + grad[3:] @ y_in_var)

constraints = []
constraints += [y_el_var >= 0, y_in_var >= 0]
constraints += [cp.sum(y_el_var) == 1.0, cp.sum(y_in_var) == 1.0]
constraints += [y_el_var[1] * tau <= B]

prob = cp.Problem(objective, constraints)
result = prob.solve()

print("grad:", grad)
print("y_el_var.value:", y_el_var.value)
# print("y_el_var_current:", y_el_var_current)
print("y_in_var.value:", y_in_var.value)
# print("y_in_var_current:", y_in_var_current)
print()

# y_el_var_current = y_el_var_current + 2/(k+2) * (y_el_var.value - y_el_var_current)
# y_in_var_current = y_in_var_current + 2/(k+2) * (y_in_var.value - y_in_var_current)

# y_iters[0:3, k] = y_el_var_current
# y_iters[3:, k] = y_in_var_current

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()

# Solver=SCS,verbose=False

In [ ]:
# y_el_var
# y_in_var
# np.hstack((y_el_var, y_in_var))

# Scratch Work:

In [3]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()


x.value: [-0.  1.]
y.value: [-0.  1.]



# Scratch Work:

In [ ]:
# # Test:

# Variables:

# y_el: \hat y_1 E, \tilde y_1 E, y_2 E
y_elig = cp.Variable(3)
# y_in: y_1 I, y_2 I
y_inel = cp.Variable(2)


# Objective:

a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])

tau = 0.5
B = 0.4

func = 0.0
func += 1/5 * a[4] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 5)
func += 1/4 * a[3] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 4)
func += 1/3 * a[2] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 3)
func += 1/2 * a[1] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 2)
func += a[0] * (y_elig[0] + y_elig[1] + y_inel[0])
func += y_inel[0] * tau / v_I + y_elig[0] * tau / v_E
func += 1/5 * a[4] * cp.power(y_elig[2] + y_inel[1], 5)
func += 1/4 * a[3] * cp.power(y_elig[2] + y_inel[1], 4)
func += 1/3 * a[2] * cp.power(y_elig[2] + y_inel[1], 3)
func += 1/2 * a[1] * cp.power(y_elig[2] + y_inel[1], 2)
func += a[0] * (y_elig[2] + y_inel[1])

objective = cp.Minimize(func)


# Constraints:

constraints = []
constraints += [y_elig >= 0, y_inel >= 0]
constraints += [cp.sum(y_elig) == 1, cp.sum(y_inel) == 1]
constraints += [y_elig[1] * tau <= B]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
print("y_elig.value:", np.round(y_elig.value, 4) )
print("y_inel.value:", np.round(y_inel.value, 4) )
print()



# power(x, p)



In [ ]:
x1 = 1
x2 = 2
print("x1:", x1, ", x2:", x2)

## CVXPY can handle 4d arrays:

In [ ]:

I, J, K, L = 2, 3, 4, 5

# Variables:
x_test = {}
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                x_test[(i, j, k, ell)] = cp.Variable(1)
            
# Objective:

func = 0.0
func += cp.sum([x_test[(i, j, k, ell)]**2 for i in range(I) for j in range(J) \
                for k in range(K) for ell in range(L)])
            
objective = cp.Minimize(func)

# Constraints:
constraints = []

for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                constraints += [cp.sum([x_test[(i, j, k, ell)] for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L) ]) == 1.0]
                constraints += [x_test[(i, j, k, ell)] >= 0.0 for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L)]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                print("i, j, k, ell:", i, j, k, ell)
                print("x_test[(i,j,k, ell)].value:", x_test[(i, j, k, ell)].value)
